Essa é uma forma de verificar como está indo as ações

In [1]:
from binance.client import Client
from secrets import api_secrets, api_key
from binance.enums import *
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from scipy import stats

client = Client(api_key, api_secrets)
client.get_account_status()

{'data': 'Normal'}

In [2]:
#Pegando ações que tenho com mais de 10 unidades
info2 = client.get_account()
lista_ativos = info2['balances']
balances = [x for x in  lista_ativos if (float(x['locked']) or float(x['free']))> 10]
busd = [x['asset'] for x in balances if x['asset'] == 'BUSD']
outras_moedas = [x for x in balances if x['asset'] != 'BUSD']
outras_moedas


[]

In [3]:
#Pior e melhor acao no momento
t = client.get_ticker()
p = [[x['symbol'] , x['priceChangePercent']] for x in t]
cabeca = sorted(p, reverse=True, key=lambda tup: float(tup[1]))
calda = sorted(p, key=lambda tup: float(tup[1]))
print(cabeca[0], calda[0])

['MIRBUSD', '27.241'] ['BTCSTBUSD', '-32.948']


In [8]:

klines = client.get_historical_klines('MIRBUSD', Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")
precos = np.array([float(x[1]) for x in klines])
preco_minimo = min(precos)
moda = stats.mode(precos)[0][0]
proc_delta_p = round(100*((precos[-1] - preco_minimo)/(preco_minimo)), 2)
dist_moda = round(100*((precos[-1] - moda)/(moda)), 2)
print(f'está à {proc_delta_p} % do preço mínimo')
print(f'está à {dist_moda} % da moda')
px.line(x = np.arange(len(precos)), y = precos)

está à 29.55 % do preço mínimo
está à 20.98 % da moda


In [68]:
def ultima_ordem(outras_moedas):
    var = []
    for y in outras_moedas:
        orders = client.get_all_orders(symbol=f'{y["asset"]}BUSD', limit=5)
        or_exe = [x for x in orders if x['status'] != 'CANCELED']
        or_exe = [x for x in or_exe if x['status'] != 'NEW']
        var.append(or_exe)
    return var
ul_ord = ultima_ordem(outras_moedas)

[[],
 [{'symbol': 'LITBUSD',
   'orderId': 157099438,
   'orderListId': -1,
   'clientOrderId': 'web_8cc2a0d37ba34fb4897f37f817ab51f4',
   'price': '0.84000000',
   'origQty': '64.60000000',
   'executedQty': '64.60000000',
   'cummulativeQuoteQty': '54.26400000',
   'status': 'FILLED',
   'timeInForce': 'GTC',
   'type': 'LIMIT',
   'side': 'BUY',
   'stopPrice': '0.00000000',
   'icebergQty': '0.00000000',
   'time': 1670538563738,
   'updateTime': 1670538563741,
   'isWorking': True,
   'workingTime': 1670538563738,
   'origQuoteOrderQty': '0.00000000',
   'selfTradePreventionMode': 'NONE'}]]